In [ ]:
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

import numpy as np
import math3d as m3d
import math
import sys
import urx
from time import sleep
from svgpathtools import svg2paths, Arc, Line, QuadraticBezier, CubicBezier

In [ ]:
np.set_printoptions(precision=3)
r = urx.Robot("192.168.0.2", use_rt=True)
r.set_tcp((0, 0, 0.15, 0, 0, 0)) # set dummy TCP, 15 cm in front of the flange
r.set_payload(0.1) # set tool payload (in kg)
sleep(0.2)

In [ ]:
a = 0.1 # max acceleration (in m/s^2)
v = 0.2 # max velocity (in m/s)
hover = 0.02 # hover height above the canvas while not painting (in m)
feed = 0.001 # brush feed while painting in m/m #default was 0.006
offset = 0.003 # initial brush feed when starting to paint in m 

paint_depth = 0.100     # depth of paint surface from the paint pot coordinate (in m) 
in_paint_duration = 0.5 # duration for brush in paint pot (in s)
drip_off_duration = 1   # duration to stay over paint pot (in s)

r.set_csys(m3d.Transform()) #set the active coordinate system(csys) for the robot to the base csys

In [ ]:
# Define Joint angles for important positions:
# Make sure that a free path exists between any of those!
j_home         = ( 0    , -math.pi/2,  0, -math.pi/2, 0, 0)
j_paint_above  = (-1.257, -1.332, -2.315, -1.065, 1.571,  0.313)
j_canvas_above = (-0.671, -1.464, -1.975,  0.026, 2.302, -0.169)
j_brush_change = (0.0, -0.725, -2.153, -1.570, 0, 0)

In [ ]:
# Defining TCP transform for each brush in the tool
brush = {"red":    (0.0903, 0.003, 0.1275, 0.1418, 0.5067, 0.6670),
         "yellow": (0.0903, 0.003, 0.1275, 0.1418, 0.5067, 0.6670),
         "blue":   (0.0903, 0.003, 0.1275, 0.1418, 0.5067, 0.6670),
         "black":  (0.0903, 0.003, 0.1275, 0.1418, 0.5067, 0.6670),
         "green":  (0.0903, 0.003, 0.1275, 0.1418, 0.5067, 0.6670),
         "white":  (0.0903, 0.003, 0.1275, 0.1418, 0.5067, 0.6670),
         "brown":  (0.0903, 0.003, 0.1275, 0.1418, 0.5067, 0.6670)
        }

In [ ]:
# Canvas coordinates (base csys):
# p0 ---------> px
# |
# |
# py                   

p0 = m3d.Transform(( 0.551,  0.224,  0.697, 2.511, -2.478, 2.063))
px = m3d.Transform(( 0.559, -0.498,  0.697, 2.511, -2.478, 2.063))
py = m3d.Transform(( 0.447, 0.229, 0.137, 2.511, -2.478, 2.063))

dx = px.pos - p0.pos
dy = py.pos - p0.pos
canvas_coordinates = m3d.Transform.new_from_xyp(dx, dy, p0.pos)

# A satisfying test
# r.set_tcp(brush['black'])
# r.movel(p0, acc = a/2, vel=v/3)
# r.movel(px, acc = a/2, vel=v/3)
# r.movel(py, acc = a/2, vel=v/3)
# r.movel(p0, acc = a/2, vel=v/3)


In [ ]:
# Paint pot coordinates:
paint = {"red":    (-0.12, -0.280, 0.08, 0, np.pi, 0),
         "yellow": (-0.04, -0.280, 0.08, 0, np.pi, 0),
         "blue":   ( 0.04, -0.280, 0.08, 0, np.pi, 0),
         "black":  ( 0.12, -0.285, 0.08, 0, np.pi, 0)}

In [ ]:
# Paint drop removal coordinates:
mesh  = {"red":    m3d.Vector(-0.12, -0.40, 0.057),
         "yellow": m3d.Vector(-0.04, -0.40, 0.055),
         "blue":   m3d.Vector( 0.04, -0.40, 0.054),
         "black":  m3d.Vector( 0.12, -0.40, 0.053)}

In [ ]:
def brush_transform(index, angle, length):
    rot = m3d.Orientation.new_rot_z(index * np.pi / 2)
    rot.rotate_x(angle)
    vec = m3d.Transform(rot, (0, 0, 0)) * m3d.Vector(0, 0, length)
    return m3d.Transform(rot, vec)

In [ ]:
# Brush calibration parameters
# Old brush holder    
#brush = {"red":    brush_transform(-1, 24 * np.pi / 180, 0.148),
#         "yellow": brush_transform( 0, 19 * np.pi / 180, 0.145),
#         "blue":   brush_transform( 1, 20 * np.pi / 180, 0.145),
#         "black":  brush_transform( 2, 24 * np.pi / 180, 0.147)}
# New brush holder
#brush = {"red":    brush_transform(-1, 32 * np.pi / 180, 0.144),
#         "yellow": brush_transform( 0, 33 * np.pi / 180, 0.148),
#         "blue":   brush_transform( 1, 30 * np.pi / 180, 0.144),
#         "black":  brush_transform( 2, 31 * np.pi / 180, 0.144)}

# Medium size brush holder
#brush = {"red":    brush_transform(-1, 25 * np.pi / 180, 0.140),
#         "yellow": brush_transform( 0, 25 * np.pi / 180, 0.140),
#         "blue":   brush_transform( 1, 26 * np.pi / 180, 0.140),
#         "black":  brush_transform( 2, 25 * np.pi / 180, 0.140)}

# Small size brush holder
brush = {"red":    brush_transform(-0.935, 28.0 * np.pi / 180, 0.140),
         "yellow": brush_transform( 0.09, 27 * np.pi / 180, 0.138),
         "blue":   brush_transform( 1, 29 * np.pi / 180, 0.139),
         "black":  brush_transform( 2.1, 29 * np.pi / 180, 0.139)}
#%%

In [ ]:
#p = r.getl()
#print "Start tool pose is: ", np.array(p)
#print "Joint space: ", np.array(r.getj()) * 180 / math.pi

#print "Move to home"
#r.movej(j_home, acc=1.0, vel=v)


In [ ]:
def move_home():
    """A wrapper function to move the robot to the j_home joints configuration"""
    print "Moving to home position"
    r.movej(j_home, acc=1.0, vel=v)

#move_home()  #uncomment and run cell to test the functions defined in this cell

In [ ]:
def move_to_canvas():
    """A wrapper function to move the robot to the j_canvas_above joints configuration"""
    print "Moving to above canvas position"
    j = r.getj() # Keep orientation of last joint
    r.movej(j_canvas_above[:5] + (j[5],), acc=a, vel=v)
    
#move_to_canvas()  #uncomment and run cell to test the functions defined in this cell

In [ ]:
def move_to_paint():
    """A wrapper function to move the robot to the j_paint_above joints configuration"""
    print "Moving to above paint position"
    r.movej(j_paint_above, acc=a, vel=v)
    #j = r.getj() # Keep orientation of last joint
    #r.movej(j_paint_above[:5] + (j[5],), acc=a, vel=v)
    
#move_to_paint()    #uncomment and run cell to test the functions defined in this cell

In [ ]:
def move_to_brush_change():
    """A wrapper function to move the robot to the j_brush_change joints configuration"""
    print "Moving to brush change position"
    r.movej(j_brush_change, acc=a, vel=v)

#move_to_brush_change()  #uncomment and run cell to test the functions defined in this cell

In [ ]:
def move_to_canvas_origin(stroke):
    print "Setting active coordinate system (csys) to canvas csys"
    r.set_csys(canvas_coordinates)
    r.set_tcp(brush[stroke])
    r.movel((0, 0, -hover, 0, 0, 0), acc=a/2, vel=v/3)

def move_to_canvas_xaxis(stroke):
    print "Setting active coordinate system (csys) to canvas csys"
    r.set_csys(canvas_coordinates)
    r.set_tcp(brush[stroke])
    r.movel((0.75, 0, -hover, 0, 0, 0), acc=a/2, vel=v/3)    
    
def move_to_canvas_yaxis(stroke):
    print "Setting active coordinate system (csys) to canvas csys"
    r.set_csys(canvas_coordinates)
    r.set_tcp(brush[stroke])
    r.movel((0, 0.55, -hover, 0, 0, 0), acc=a/2, vel=v/3)

    
#move_to_canvas_origin('blue')    #uncomment and run cell to test the functions defined in this cell

In [ ]:
def get_paint(stroke):
    print "    Fetching", stroke, "Paint:"
    
    # stroke being the color string: eg 'red'
    print "      Setting active coordinate system (csys) to base csys"
    r.set_csys(m3d.Transform())

    # TODO: check current position
    print "      Distance to pots:", r._get_joints_dist(j_paint_above)

    print "      Getting new paint"
    #   Move with no brush selected to avoid extreme rotations of last joint
    r.set_tcp((0, 0, 0.15, 0, 0, 0))
    print "      Moving to paint pot coordinate"
    r.movel(paint[stroke], acc=a, vel=v)
    # TODO:  Measure color depth
    
    #   Select brush
    r.set_tcp(brush[stroke])
    r.movel(paint[stroke], acc=a, vel=v)
    
    #   Move into color
    print "      Moving into paint pot"
    r.down(z=paint_depth, acc=a, vel=v)
    sleep(in_paint_duration)

    print "      Moving back to paint pot coordinate"
    r.up(z=paint_depth, acc=a, vel=v)
    print "      Waiting for color to drip off"
    sleep(drip_off_duration)
    
    print "      Removing excess paint from tip of brush"
    radius = 0.018
    circle = [m3d.Transform(m3d.Orientation.new_rot_z(i * np.pi / 6), (0, 0, 0)) * m3d.Vector(-radius, -radius, 0) for i in range(8)]
    circle = [m3d.Transform((0, np.pi, 0), mesh[stroke] + c) for c in circle]
    circle.append(m3d.Transform(paint[stroke]))
    r.movels(circle, acc=a, vel=v/4)

    
#get_paint('beige')

In [ ]:
def paint_path(path):
    print "      Setting active coordinate system (csys) to canvas csys"
    r.set_csys(canvas_coordinates)

    # TODO: check current position
    print "      Distance to canvas:", r._get_joints_dist(j_canvas_above)

    print "    Painting path"
    
    j = 0 #subpath counter
    
    for sub in path.continuous_subpaths():
        print '    SUB-PATH', j, '---------------------------------------------------------------'
        print '        Length: %s mm' % (round(sub.length()))
        print '        Number of segments', ': ', len(sub)
        
        r.movel((sub.start.real / 1e3, sub.start.imag / 1e3, -hover, 0, 0, 0), acc=a, vel=v)
        poses = []
        acc_dist = 0
        
        k = 0   #segment counter
        for seg in sub:
            if isinstance(seg, Line):
                print "        SEGMENT", k, ':', 'Line (%s mm)' % (round(seg.length()))
                print seg.start.real, seg.start.imag
                poses.append((seg.start.real / 1e3, seg.start.imag / 1e3, offset + feed * acc_dist / 1e3, 0, 0, 0))
                acc_dist += seg.length()
            
            elif isinstance(seg, Arc):
                print "        SEGMENT", k, ':', 'Arc (%s mm) '% (round(seg.length())) , "TODO: Arc is beta"
                #print "TODO: Arc is not supported, Line is used"
                #poses.append((seg.start.real / 1e3, seg.start.imag / 1e3, offset + feed * acc_dist / 1e3, 0, 0, 0))
                #print seg
                
                step = min(10.0 / seg.length(), 0.5) # one point everz 10mm but at least two points
                points = [seg.point(t) for t in np.arange(0, 1, step)]
                poses.extend([(p.real / 1e3, p.imag / 1e3, offset + feed * acc_dist / 1e3, 0, 0, 0) for p in points])
                
                #This interactive demo might help you understand the concepts behind SVG arcs: http://codepen.io/lingtalfi/pen/yaLWJG (tested in chrome and firefox only, might not work in your browser)
                #print seg
                #print poses
                # TODO acc_dist should be incremented from point to point
                
            elif isinstance(seg, QuadraticBezier):
                print "        SEGMENT", k, ':', 'QuadraticBezier (%s mm) '% (round(seg.length())) , "TODO: QuadraticBezier is beta"
                step = min(10.0 / seg.length(), 0.5) # one point every 10mm but at least two points
                points = [seg.point(t) for t in np.arange(0, 1, step)]
                poses.extend([(p.real / 1e3, p.imag / 1e3, offset + feed * acc_dist / 1e3, 0, 0, 0) for p in points])
                # TODO acc_dist should be incremented from point to point

            elif isinstance(seg, CubicBezier):
                print "        SEGMENT", k, ':', 'CubicBezier (%s mm) '% (round(seg.length())) , "TODO: CubicBezier is beta"
                step = min(10.0 / seg.length(), 0.5) # one point everz 10mm but at least two points
                points = [seg.point(t) for t in np.arange(0, 1, step)]
                poses.extend([(p.real / 1e3, p.imag / 1e3, offset + feed * acc_dist / 1e3, 0, 0, 0) for p in points])
                # TODO acc_dist should be incremented from point to point
                
            k+=1 #incrementing segment counter

        poses.append((sub.end.real / 1e3, sub.end.imag / 1e3, offset, 0, 0, 0))
        poses.append((sub.end.real / 1e3, sub.end.imag / 1e3, -hover, 0, 0, 0))        
        r.movels(poses, acc=a, vel=v/4, radius = 0.00, threshold=0.001)
        
        j+=1 #incrementing subpath counter
        
    # If we are on left side of canvas move to safe position first
    r.movel((0.6, 0.3, -hover, 0, 0, 0), acc=a, vel=v)

In [ ]:
def paint_svg(paths, attributes):
    i = 0 #path counter
    print 'Total number of paths to be painted:', len(paths)
    
    for (path, attr) in zip(paths, attributes):
        stroke = attr['stroke']
        path_id = attr['id']
        #paths_to_paint = ['polyline98'] # a list of strings containing path_id of paths to be painted
        
        if True: #stroke=='white'and path_id in paths_to_paint:  
            print 'PATH', i, ', PATH ID:', path_id, ' Color:', stroke '-----------------------------------------------------------------------'
            #print '    Path is closed:', path._closed
            print '    Number of sub-paths:', len(path.continuous_subpaths())
            
            move_to_paint()
            try:
                get_paint(stroke)
                move_to_canvas()
                paint_path(path)
            except Exception as e:
                print "ERROR:", e
                raw_input("Press enter to continue... ")
            
            # This is IMPORTANT to prevent embarassing smudges in the event a particular path fails and 
            # you resume painting after an exception is raised
            r.movel((0.6, 0.3, -hover, 0, 0, 0), acc=a, vel=v)

            i += 1 #incrementing path counter

    print "Hurray! Painting complete!"

In [ ]:
paths_all, attributes_all = svg2paths('paintings/svg/peter_inkscape_Flame.svg')

In [ ]:
#unpainted = [20]
#paths = [paths_all[i] for i in range(5,26)]
#attributes = [attributes_all[i] for i in range(5,26)
#paint_svg(paths, attributes)

num_paths = len(paths_all)
paths = [paths_all[i] for i in range(num_paths)]  #range(num_paths)
attributes = [attributes_all[i] for i in range(num_paths)]  #range(num_paths)
paint_svg(paths, attributes)

In [ ]:
# This would be the full program

try:
    move_home()
    paths, attributes = svg2paths('190905 JoPostmandarkblue.svg') #painting to paint       
    #paths, attributes = svg2paths(sys.argv[1])
    paint_svg(paths, attributes)
except Exception as e:
    print "ERROR:", e
    raw_input("Press enter to continue... ")
    

move_to_canvas()
move_home()

#%%

#r.stopj()
#p = r.getl()
#print "Tool pose is: ", np.array(p)
#print "Robot joints: ", r.getj()

#    r.secmon.close()
#    r.rtmon.close()
#    r = None
#    sleep(2)

In [ ]:
from IPython.display import SVG
def show_svg():
    return SVG("JoPostmangreen.svg")

show_svg()

In [ ]:
# Always remember to close the communication to the robot and have not more than one instances running
r.secmon.close()
r.rtmon.close()
r = None
sleep(2)